# Part 3 - Communicating with an external agent

**!!! You should start this part in the [external agent notebook](http://localhost:8889/lab/workspaces/auto-w/tree/Configure%20External%20Agent.ipynb)**

### Initialise the multitenant controller

In [ ]:
%autoawait
import time
import asyncio
import pprint
from aiohttp import ClientResponseError

from aries_basic_controller.aries_tenant_controller import AriesTenantController

# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://multitenant-agent:8021"


### Updating JWT of the agent controller

Retrieve Alice's token we have stored previously

In [ ]:
%store -r alice_jwt
%store -r wallet_id_alice

In [ ]:
print(alice_jwt)

In [ ]:
# Based on the aca-py agent you wish to control
agent_controller = AriesTenantController(admin_url=ADMIN_URL, wallet_id=wallet_id_alice, tenant_jwt=alice_jwt)

Now we can update the agent controller with the JWT Token

In [ ]:
agent_controller.update_tenant_jwt(alice_jwt, wallet_id_alice)

Let's check it's really there

In [ ]:
print(agent_controller.tenant_jwt)

In [ ]:
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)

### Go to the [external agent](http://localhost:8889/lab/workspaces/auto-w/tree/Configure%20External%20Agent.ipynb) before you continue, if you haven't already generated an invitation



### Accept Invite From external agent

Replace the invitation object below with the one you have generated in the mediator notebook

In [ ]:
external_invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '83e7456b-0f15-42d9-80b5-2074d57c54ba', 'serviceEndpoint': 'https://16d921860d17.ngrok.io', 'label': 'EXTERNAL', 'recipientKeys': ['HTJ6VPVJ4XRhJnxiAK9EziTHs27tcyMkajqLLFEUBLBq']}

In [ ]:
response = await agent_controller.connections.accept_connection(external_invitation)
pp.pprint(response)

In [ ]:
connection_id = response["connection_id"]
print(connection_id)

In [ ]:
### get the connection
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)

In [ ]:
# Let's check for the state
def check_connection(connection):
    if connection['state'] != 'active':
        print("No active connection. \n\nPlease go back and ensure you have established an active connection between the mediator agent and Alice's subwallet agent\n")    
        print("State: " + connection['state']+ "\n")    
    else:
        print("Active connection established\n")
        print("State: " + connection['state']+ "\n")    
        pp.pprint(connection)

check_connection(connection)

### Great! Well done, if you made it here. Head back to the notebook of the [external agent](http://localhost:8889/lab/workspaces/auto-w/tree/Configure%20External%20Agent.ipynb) and issue the credential.

### Let's have a look at the records

This should give us some results and our submitted record with the credentials for whomever you created in the external notebook should be in there. 

In [ ]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (external)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

### Request Credential from Issuer
If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the offer_received state

In [ ]:
try:
    record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
    state = record['state']
    role = record['role']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
except ClientResponseError as err:
    print(err)

### Store the credential
Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state

In [ ]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

In [ ]:
try:
    response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
    state = response['state']
    role = response['role']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
except ClientResponseError as err:
    print(err)

### Great. You're done with this tutorial. 

Almost - just terminate the controller below.

In [ ]:
await agent_controller.terminate()